# Notebook 03: Robustez y Validacion para Pipelines de AI/ML

**Modulo:** 01 - Programacion Python  
**Nivel:** Avanzado (el notebook mas completo del modulo 01)  
**Tiempo estimado:** 90 - 120 minutos  
**Temas cubiertos:** `05_errores_y_debug` | `08_excepciones_avanzadas` | `09_generadores` | `10_comprehensions` | `11_logging` | `12_pydantic`

---

## Objetivos de aprendizaje

Al finalizar este notebook seras capaz de:

1. Manejar errores de forma estructurada con `try / except / else / finally`.
2. Crear **excepciones custom** con jerarquia para pipelines de ML.
3. Usar **generadores** para procesamiento lazy de grandes volumenes de datos.
4. Aplicar **comprehensions** (list, dict, set) de forma correcta y eficiente.
5. Configurar **logging profesional** con niveles, handlers y formatters.
6. Validar datos de entrada y salida con **Pydantic** antes de alimentar modelos.

---

### Por que importa en AI Engineering

Un modelo de ML puede ser excelente, pero si el pipeline que lo alimenta:
- no valida datos de entrada,
- no maneja errores de forma explicita,
- no registra que paso en cada etapa,

...entonces el sistema completo es **fragil**. Este notebook te da las herramientas para construir pipelines **robustos**.

In [ ]:
# ============================================================
# Setup y verificacion del entorno
# ============================================================
import sys
import time
import logging
import os
import tempfile
import collections
import functools
from typing import List, Dict, Optional, Generator, Any, Literal
from io import StringIO

# Pydantic (ya incluido en el proyecto)
try:
    import pydantic
    _pydantic_ok = True
except ImportError:
    _pydantic_ok = False

print("=" * 50)
print("  Verificacion del entorno")
print("=" * 50)
print(f"  Python  : {sys.version.split()[0]}")
print(f"  Pydantic: {pydantic.__version__ if _pydantic_ok else 'NO INSTALADO'}")
print("=" * 50)

if not _pydantic_ok:
    print("\n[!] Instala pydantic: pip install pydantic")

---

## Seccion 1: Manejo de Errores con `try / except / else / finally`

### Flujo de ejecucion

```
        try:
          codigo_riesgoso()
              |
     +--------+--------+
     |                  |
  EXCEPCION?         SIN ERROR
     |                  |
  except:            else:
  manejar error      codigo si todo ok
     |                  |
     +--------+--------+
              |
          finally:
          SIEMPRE se ejecuta
          (cleanup, cerrar archivos, etc.)
```

### Excepciones comunes en Python

| Excepcion            | Causa tipica                                 | Ejemplo en AI/ML                        |
|----------------------|----------------------------------------------|-----------------------------------------|
| `TypeError`          | Tipo incorrecto en operacion                 | Pasar string donde se espera int        |
| `ValueError`         | Valor fuera de rango o formato invalido      | `temperature = -1.0`                    |
| `KeyError`           | Clave no existe en diccionario               | Acceder a feature que no existe          |
| `FileNotFoundError`  | Archivo/directorio no encontrado             | Ruta de modelo inexistente              |
| `IndexError`         | Indice fuera de rango                        | Acceder a batch vacio                   |
| `AttributeError`     | Objeto no tiene el atributo                  | Modelo sin metodo `.predict()`          |
| `RuntimeError`       | Error generico en tiempo de ejecucion        | Fallo en GPU / framework                |
| `StopIteration`      | Iterador agotado                             | Generador sin mas datos                 |

**Regla de oro:** Captura excepciones **especificas**, nunca uses `except:` sin tipo.

In [ ]:
# ============================================================
# Ejemplo completo: try / except / else / finally
# Caso practico: cargar configuracion de un modelo
# ============================================================
import json

def cargar_config_modelo(ruta: str) -> dict:
    """Carga configuracion JSON de un modelo de forma segura."""
    archivo = None
    try:
        archivo = open(ruta, "r", encoding="utf-8")
        config = json.load(archivo)
    except FileNotFoundError:
        print(f"  [ERROR] Archivo no encontrado: {ruta}")
        print("  -> Usando configuracion por defecto")
        config = {"model_name": "default", "max_tokens": 256}
    except json.JSONDecodeError as e:
        print(f"  [ERROR] JSON invalido en {ruta}: {e}")
        config = {"model_name": "default", "max_tokens": 256}
    else:
        # Solo se ejecuta si NO hubo excepcion
        print(f"  [OK] Configuracion cargada desde {ruta}")
        print(f"  -> Claves encontradas: {list(config.keys())}")
    finally:
        # SIEMPRE se ejecuta: limpieza
        if archivo is not None:
            archivo.close()
            print("  [CLEANUP] Archivo cerrado correctamente")
    return config


# --- Demo 1: archivo que no existe ---
print("Caso 1: archivo inexistente")
print("-" * 40)
config1 = cargar_config_modelo("/ruta/que/no/existe.json")
print(f"  Resultado: {config1}\n")

# --- Demo 2: archivo valido (creamos uno temporal) ---
print("Caso 2: archivo JSON valido")
print("-" * 40)
with tempfile.NamedTemporaryFile(mode="w", suffix=".json", delete=False) as f:
    json.dump({"model_name": "gpt-mini", "max_tokens": 512, "temperature": 0.7}, f)
    ruta_temp = f.name

config2 = cargar_config_modelo(ruta_temp)
print(f"  Resultado: {config2}\n")
os.unlink(ruta_temp)  # limpiar archivo temporal

# --- Demo 3: JSON corrupto ---
print("Caso 3: JSON corrupto")
print("-" * 40)
with tempfile.NamedTemporaryFile(mode="w", suffix=".json", delete=False) as f:
    f.write("{modelo: sin comillas}")
    ruta_corrupta = f.name

config3 = cargar_config_modelo(ruta_corrupta)
print(f"  Resultado: {config3}")
os.unlink(ruta_corrupta)

### Custom Exceptions para AI/ML

**Cuando crear excepciones propias:**
- Cuando los errores built-in no describen bien la falla de tu dominio.
- Cuando necesitas que distintos componentes del pipeline capturen errores de forma selectiva.
- Cuando quieres incluir contexto adicional (nombre del modelo, ID del registro, etc.).

**Diseno de jerarquia:**

```
  Exception
      |
  PipelineError          <- base de todos nuestros errores
      |
      +-- DataValidationError   <- datos de entrada invalidos
      +-- ModelLoadError        <- fallo al cargar modelo
      +-- InferenceError        <- fallo durante prediccion
```

Con esta jerarquia puedes capturar `PipelineError` para atrapar **todos** los errores del pipeline, o capturar subclases para manejar cada caso por separado.

In [ ]:
# ============================================================
# Jerarquia de excepciones custom para un pipeline de ML
# ============================================================

class PipelineError(Exception):
    """Error base para todas las fallas del pipeline."""
    def __init__(self, message: str, component: str = "unknown"):
        self.component = component
        super().__init__(f"[{component}] {message}")


class DataValidationError(PipelineError):
    """Error cuando los datos de entrada no pasan validacion."""
    def __init__(self, message: str, field: str = "", value: Any = None):
        self.field = field
        self.value = value
        detail = f"{message} (field={field!r}, value={value!r})"
        super().__init__(detail, component="DataValidation")


class ModelLoadError(PipelineError):
    """Error al cargar un modelo desde disco o registro."""
    def __init__(self, model_name: str, reason: str):
        self.model_name = model_name
        super().__init__(f"No se pudo cargar '{model_name}': {reason}", component="ModelLoader")


class InferenceError(PipelineError):
    """Error durante la prediccion / inferencia."""
    def __init__(self, message: str, request_id: str = ""):
        self.request_id = request_id
        super().__init__(f"{message} (request_id={request_id!r})", component="Inference")


# --- Demo: lanzar y capturar cada tipo ---
def demo_excepciones():
    errores = [
        DataValidationError("Valor negativo", field="age", value=-5),
        ModelLoadError("sentiment-v2", reason="archivo corrupto"),
        InferenceError("Timeout en GPU", request_id="req-42"),
    ]

    for error in errores:
        try:
            raise error
        except DataValidationError as e:
            print(f"  DATOS INVALIDOS -> {e}  | field={e.field}")
        except ModelLoadError as e:
            print(f"  MODELO FALLO    -> {e}  | model={e.model_name}")
        except InferenceError as e:
            print(f"  INFERENCIA FALLO-> {e}  | req_id={e.request_id}")

    # --- Exception chaining con 'raise ... from ...' ---
    print("\n--- Exception chaining ---")
    try:
        try:
            data = {"age": "no es numero"}
            int(data["age"])
        except ValueError as original:
            raise DataValidationError(
                "No se pudo convertir a entero",
                field="age",
                value=data["age"]
            ) from original
    except PipelineError as e:
        print(f"  Capturado: {e}")
        print(f"  Causa original: {e.__cause__}")
        print(f"  Tipo: {type(e).__name__}")

demo_excepciones()

---

## Seccion 2: Generadores - Procesamiento Lazy

### Lista vs Generador en memoria

```
  LISTA (eager)                         GENERADOR (lazy)
  ============                          ================

  [0, 1, 2, ..., 999_999]              (x for x in range(1_000_000))
  ^^^^^^^^^^^^^^^^^^^^^^^^              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  Todos los elementos viven             Solo el SIGUIENTE elemento
  en memoria al mismo tiempo            se computa cuando se pide

  RAM: ~8 MB (1M ints)                  RAM: ~120 bytes (constante)

  Pro: acceso aleatorio (lista[500])    Pro: memoria constante O(1)
  Con: memoria crece con N             Con: solo iteracion hacia adelante
```

### `yield` vs `return`

| Aspecto          | `return`                      | `yield`                           |
|------------------|-------------------------------|-----------------------------------|
| Funcion termina? | Si, inmediatamente            | No, se **pausa** y se puede resumir|
| Devuelve         | Un valor unico                | Un valor cada vez que se llama    |
| Tipo resultado   | El tipo del valor             | Un objeto `generator`             |
| Memoria          | Proporcional al resultado     | Constante O(1)                    |
| Uso tipico       | Calcular y devolver           | Iterar sobre secuencias grandes   |

In [ ]:
# ============================================================
# Generadores: basicos y batch_iterator
# ============================================================

# --- Generador basico con yield ---
def contar_hasta(n: int) -> Generator[int, None, None]:
    """Genera numeros de 1 a n."""
    i = 1
    while i <= n:
        yield i
        i += 1

gen = contar_hasta(4)

print("Tipo del generador:", type(gen))
print("next():", next(gen))  # 1
print("next():", next(gen))  # 2
print("Resto con for:")
for valor in gen:  # continua donde se quedo: 3, 4
    print(f"  {valor}")

# StopIteration al agotar
try:
    next(gen)
except StopIteration:
    print("\nStopIteration: generador agotado")


# --- Generador practico: batch_iterator ---
print("\n" + "=" * 50)
print("batch_iterator: partir datos en lotes")
print("=" * 50)

def batch_iterator(data: list, batch_size: int) -> Generator[list, None, None]:
    """Genera lotes (sublistas) de tamanio batch_size."""
    for i in range(0, len(data), batch_size):
        yield data[i : i + batch_size]


# Simular un dataset de 12 registros
dataset = [f"registro_{i:03d}" for i in range(12)]
print(f"Dataset total: {len(dataset)} registros\n")

for batch_num, batch in enumerate(batch_iterator(dataset, batch_size=4), start=1):
    print(f"  Batch {batch_num}: {batch}")

In [ ]:
# ============================================================
# Comparacion de memoria: lista vs generador
# ============================================================

N = 1_000_000

# Enfoque 1: lista completa en memoria
lista_completa = [x ** 2 for x in range(N)]

# Enfoque 2: generador (lazy)
generador = (x ** 2 for x in range(N))

mem_lista = sys.getsizeof(lista_completa)
mem_gen = sys.getsizeof(generador)

print("=" * 55)
print(f"  Comparacion de memoria: {N:,} elementos")
print("=" * 55)
print(f"  {'Estructura':<20} {'Memoria':>15} {'Ratio':>10}")
print(f"  {'-'*20} {'-'*15} {'-'*10}")
print(f"  {'Lista':<20} {mem_lista:>12,} B {'1.0x':>10}")
print(f"  {'Generador':<20} {mem_gen:>12,} B {f'{mem_lista/mem_gen:.0f}x menos':>10}")
print("=" * 55)
print(f"\n  La lista usa ~{mem_lista / mem_gen:.0f}x mas memoria que el generador.")
print("  El generador NO almacena los resultados; los computa bajo demanda.")

### Generator Pipelines (cadenas de generadores)

Los generadores se pueden **encadenar** para crear pipelines de procesamiento donde los datos fluyen de uno a otro **sin materializar resultados intermedios**.

```
  Datos crudos
      |
      v
  [read_lines]    -- yield linea por linea
      |
      v
  [filter_valid]  -- yield solo lineas validas
      |
      v
  [parse_records] -- yield dicts parseados
      |
      v
  [aggregate]     -- consume y acumula resultado final

  Memoria usada: O(1) en cada etapa (solo 1 elemento a la vez)
```

Esto es especialmente util cuando procesas archivos grandes de features, logs de inferencia, etc.

In [ ]:
# ============================================================
# Generator pipeline: procesamiento lazy en cadena
# ============================================================

# Datos crudos simulados (como si vinieran de un archivo CSV)
datos_crudos = [
    "user_id,edad,score",    # header
    "u001,25,0.85",
    "u002,,0.72",            # edad vacia -> invalido
    "u003,30,0.91",
    "CORRUPTO",              # linea corrupta
    "u004,22,0.65",
    "u005,45,0.88",
    "u006,-3,0.50",          # edad negativa -> invalido
    "u007,35,0.77",
]


def read_lines(data: list) -> Generator[str, None, None]:
    """Etapa 1: emite lineas saltando el header."""
    for linea in data[1:]:
        yield linea


def filter_valid(lines: Generator) -> Generator[str, None, None]:
    """Etapa 2: filtra lineas que tengan exactamente 3 campos."""
    for linea in lines:
        campos = linea.split(",")
        if len(campos) == 3 and all(c.strip() for c in campos):
            yield linea


def parse_records(lines: Generator) -> Generator[dict, None, None]:
    """Etapa 3: convierte cada linea en un diccionario tipado."""
    for linea in lines:
        uid, edad_str, score_str = linea.split(",")
        edad = int(edad_str)
        if edad < 0:
            continue  # filtrar edades invalidas
        yield {"user_id": uid, "edad": edad, "score": float(score_str)}


def aggregate(records: Generator) -> dict:
    """Etapa 4 (terminal): calcula estadisticas agregadas."""
    total = 0
    suma_score = 0.0
    suma_edad = 0
    for record in records:
        total += 1
        suma_score += record["score"]
        suma_edad += record["edad"]
    if total == 0:
        return {"total": 0, "avg_score": 0, "avg_edad": 0}
    return {
        "total": total,
        "avg_score": round(suma_score / total, 3),
        "avg_edad": round(suma_edad / total, 1),
    }


# --- Encadenar el pipeline ---
pipeline = aggregate(
    parse_records(
        filter_valid(
            read_lines(datos_crudos)
        )
    )
)

print("Resultado del pipeline:")
print(f"  Registros validos : {pipeline['total']}")
print(f"  Score promedio    : {pipeline['avg_score']}")
print(f"  Edad promedio     : {pipeline['avg_edad']}")
print(f"\n  (De {len(datos_crudos) - 1} lineas de datos, {pipeline['total']} pasaron todas las etapas)")

---

## Seccion 3: Comprehensions vs Loops

### Cuando usar cada uno

| Situacion                              | Recomendacion       | Razon                                |
|----------------------------------------|---------------------|--------------------------------------|
| Transformar todos los elementos         | **Comprehension**   | Mas conciso, mas rapido              |
| Filtrar + transformar (1 condicion)     | **Comprehension**   | Legible en una linea                 |
| Logica compleja (try/except, multiples ifs) | **Loop**       | Comprehension se vuelve ilegible     |
| Efectos secundarios (print, log, append) | **Loop**          | Comprehension no debe tener side effects |
| Crear dict/set desde otra estructura    | **Comprehension**   | Dict/set comprehension es muy claro  |
| Iterar sin guardar resultado            | **Loop**            | No crees una lista solo para descartarla |

**Anti-patron:**
```python
# MAL: comprehension solo por side effects
[print(x) for x in data]   # crea lista de None, desperdicia memoria

# BIEN: loop para side effects
for x in data:
    print(x)
```

In [ ]:
# ============================================================
# Comprehensions: list, dict, set + comparacion de rendimiento
# ============================================================

# --- Ejemplos basicos ---
features = ["age", "income", "tenure", "score", "age", "income"]
valores = [25, 50000, 12, 0.85, 30, 60000]

# List comprehension: elevar al cuadrado
cuadrados = [v ** 2 for v in valores if isinstance(v, int)]
print(f"List comprehension (cuadrados de ints): {cuadrados}")

# Dict comprehension: nombre_feature -> longitud
feature_lengths = {feat: len(feat) for feat in set(features)}
print(f"Dict comprehension (feature -> len): {feature_lengths}")

# Set comprehension: features unicos
features_unicos = {feat.upper() for feat in features}
print(f"Set comprehension (unicos upper): {features_unicos}")


# --- Comparacion de rendimiento: loop vs comprehension ---
print("\n" + "=" * 55)
print("  Rendimiento: loop vs comprehension (100,000 elementos)")
print("=" * 55)

N = 100_000
datos = list(range(N))

# Enfoque 1: loop clasico
t0 = time.perf_counter()
resultado_loop = []
for x in datos:
    if x % 3 == 0:
        resultado_loop.append(x * x + 2 * x + 1)
t_loop = time.perf_counter() - t0

# Enfoque 2: list comprehension
t0 = time.perf_counter()
resultado_comp = [x * x + 2 * x + 1 for x in datos if x % 3 == 0]
t_comp = time.perf_counter() - t0

assert resultado_loop == resultado_comp, "Los resultados deben ser iguales"

print(f"  {'Enfoque':<25} {'Tiempo (ms)':>12} {'Speedup':>10}")
print(f"  {'-'*25} {'-'*12} {'-'*10}")
print(f"  {'Loop + append':<25} {t_loop*1000:>12.2f} {'1.00x':>10}")
print(f"  {'List comprehension':<25} {t_comp*1000:>12.2f} {f'{t_loop/t_comp:.2f}x':>10}")
print(f"\n  Elementos resultantes: {len(resultado_comp):,}")

---

## Seccion 4: Logging Profesional

### Los 5 niveles de logging

| Nivel      | Valor | Cuando usarlo                                         | Ejemplo en AI/ML                       |
|------------|-------|-------------------------------------------------------|----------------------------------------|
| `DEBUG`    | 10    | Detalle fino para diagnostico                         | Forma del tensor en cada capa          |
| `INFO`     | 20    | Confirmacion de que todo funciona                     | "Modelo cargado en 2.3s"              |
| `WARNING`  | 30    | Algo inesperado, pero el sistema sigue               | "GPU no disponible, usando CPU"        |
| `ERROR`    | 40    | Fallo que impide una operacion                        | "No se pudo procesar el batch 42"      |
| `CRITICAL` | 50    | Fallo total del sistema                               | "Base de datos de features inalcanzable" |

### Arquitectura del logging

```
  Tu codigo
      |
      v
  Logger (nombre unico, ej: 'ml_pipeline')
      |
      +---> Handler 1: StreamHandler  --> consola
      |         |
      |     Formatter: '%(asctime)s - %(levelname)s - %(message)s'
      |
      +---> Handler 2: FileHandler    --> archivo.log
                |
            Formatter: (puede ser diferente)
```

**Regla importante:** Nunca uses `print()` para diagnostico en produccion. Usa `logging`.

In [ ]:
# ============================================================
# Logging profesional: configuracion con handlers
# ============================================================

# Crear un logger dedicado (NO usar el root logger)
logger = logging.getLogger("ml_pipeline_demo")
logger.setLevel(logging.DEBUG)  # nivel mas bajo; los handlers filtran

# Limpiar handlers previos (por si re-ejecutas la celda)
logger.handlers.clear()

# Handler 1: consola (solo INFO y superiores)
console_handler = logging.StreamHandler(sys.stdout)
console_handler.setLevel(logging.INFO)
console_fmt = logging.Formatter("%(asctime)s | %(levelname)-8s | %(message)s", datefmt="%H:%M:%S")
console_handler.setFormatter(console_fmt)

# Handler 2: StringIO (captura todo, incluyendo DEBUG)
log_capture = StringIO()
string_handler = logging.StreamHandler(log_capture)
string_handler.setLevel(logging.DEBUG)
string_fmt = logging.Formatter("%(levelname)-8s | %(name)s | %(message)s")
string_handler.setFormatter(string_fmt)

logger.addHandler(console_handler)
logger.addHandler(string_handler)

# --- Demo de los 5 niveles ---
print("--- Mensajes visibles en consola (>= INFO) ---\n")

logger.debug("Forma del input: (32, 768)")
logger.info("Modelo cargado exitosamente en %.2f segundos", 2.34)
logger.warning("GPU no detectada, fallback a CPU")
logger.error("Batch %d fallo: datos incompletos", 42)
logger.critical("Conexion a base de features PERDIDA")

# Mostrar lo que capturo el string handler (incluye DEBUG)
print("\n--- Captura completa (incluye DEBUG) ---")
print(log_capture.getvalue())

# --- Anti-patron vs patron correcto ---
print("--- Anti-patron vs patron correcto ---\n")

modelo = "gpt-mini"
latencia = 0.342

# MAL: f-string se evalua SIEMPRE, incluso si el nivel esta desactivado
# logger.debug(f"Modelo {modelo} respondio en {latencia:.3f}s")  # <-- NO
print("  MAL:  logger.debug(f'Modelo {modelo} respondio en {latencia:.3f}s')")
print("        -> El f-string se evalua SIEMPRE aunque DEBUG este desactivado\n")

# BIEN: lazy formatting con % -- solo se evalua si el mensaje se emite
# logger.debug("Modelo %s respondio en %.3fs", modelo, latencia)  # <-- SI
print("  BIEN: logger.debug('Modelo %s respondio en %.3fs', modelo, latencia)")
print("        -> Se evalua SOLO si el nivel DEBUG esta activo")

---

## Seccion 5: Pydantic - Validacion de Datos

### Por que Pydantic en AI/ML

En pipelines de ML, la mayoria de los bugs **no vienen del modelo**, sino de:
- Datos de entrada mal formateados (tipos incorrectos, campos faltantes).
- Valores fuera de rango que el modelo no sabe manejar.
- Contratos rotos entre componentes del sistema.

**Pydantic** resuelve esto con:

| Concepto         | Que hace                                             |
|------------------|------------------------------------------------------|
| `BaseModel`      | Clase base que valida datos al instanciar            |
| `Field(...)`     | Define restricciones: min, max, default, descripcion |
| `field_validator` | Validacion custom con logica de negocio             |
| `ValidationError`| Excepcion estructurada con detalle de cada error     |

### Flujo

```
  Datos crudos (dict, JSON, API request)
      |
      v
  PydanticModel(**datos)
      |
      +-- Valido?   SI --> objeto tipado con autocompletado
      |               
      +-- Invalido? NO --> ValidationError con lista de errores
```

In [ ]:
# ============================================================
# Pydantic: modelos de validacion para AI/ML
# ============================================================
from pydantic import BaseModel, Field, ValidationError, field_validator


class InferenceRequest(BaseModel):
    """Modelo de request para un endpoint de inferencia."""
    prompt: str = Field(min_length=5, max_length=2000, description="Texto de entrada")
    max_tokens: int = Field(default=256, ge=1, le=4096, description="Tokens maximos")
    temperature: float = Field(default=0.2, ge=0.0, le=2.0, description="Creatividad")
    model: str = Field(default="gpt-mini", description="Modelo a usar")


class FeatureRow(BaseModel):
    """Modelo para una fila de features de un dataset de ML."""
    user_id: str = Field(min_length=1, description="ID unico del usuario")
    age: int = Field(ge=0, le=120, description="Edad en anios")
    monthly_income_usd: float = Field(ge=0, description="Ingreso mensual en USD")
    tenure_months: int = Field(ge=0, description="Meses como cliente")

    @field_validator("monthly_income_usd")
    @classmethod
    def reject_unrealistic_income(cls, value: float) -> float:
        if value > 1_000_000:
            raise ValueError("monthly_income_usd parece fuera de rango realista (> 1M)")
        return value


# --- Demo 1: datos validos ---
print("=" * 55)
print("  Datos VALIDOS")
print("=" * 55)

req = InferenceRequest(prompt="Explica que es machine learning", temperature=0.7)
print(f"  InferenceRequest: {req.model_dump()}")

fila = FeatureRow(user_id="u001", age=28, monthly_income_usd=4500.0, tenure_months=18)
print(f"  FeatureRow: {fila.model_dump()}")


# --- Demo 2: datos invalidos ---
print(f"\n{'=' * 55}")
print("  Datos INVALIDOS (Pydantic captura cada error)")
print("=" * 55)

# Request invalido
try:
    InferenceRequest(
        prompt="Hi",         # muy corto (min_length=5)
        max_tokens=-10,       # negativo (ge=1)
        temperature=5.0,      # fuera de rango (le=2.0)
    )
except ValidationError as e:
    print(f"\n  InferenceRequest - {e.error_count()} errores:")
    for err in e.errors():
        print(f"    - Campo: {err['loc']} | Tipo: {err['type']} | Msg: {err['msg']}")

# Feature row invalida
try:
    FeatureRow(
        user_id="",               # vacio (min_length=1)
        age=200,                   # fuera de rango (le=120)
        monthly_income_usd=5_000_000,  # validador custom
        tenure_months=-5,          # negativo (ge=0)
    )
except ValidationError as e:
    print(f"\n  FeatureRow - {e.error_count()} errores:")
    for err in e.errors():
        print(f"    - Campo: {err['loc']} | Tipo: {err['type']} | Msg: {err['msg']}")

In [ ]:
# ============================================================
# Comparacion: validacion con dict vs Pydantic
# ============================================================

def fake_model_predict(age: int, income: float, tenure: int) -> float:
    """Simula un modelo que espera tipos y rangos correctos."""
    if not isinstance(age, int):
        raise TypeError(f"age debe ser int, recibido {type(age).__name__}")
    # Calcular un "score" ficticio
    return round(min(1.0, (age * 0.01 + income * 0.00001 + tenure * 0.005)), 3)


# --- Enfoque 1: dict sin validacion ---
print("=" * 55)
print("  Enfoque 1: Dict sin validacion")
print("=" * 55)

datos_malos = {
    "user_id": "u099",
    "age": "veintiocho",          # string en vez de int!
    "monthly_income_usd": 4500,
    "tenure_months": 12,
}

# El dict NO valida nada. El error aparece DESPUES, lejos del origen.
try:
    score = fake_model_predict(
        age=datos_malos["age"],
        income=datos_malos["monthly_income_usd"],
        tenure=datos_malos["tenure_months"],
    )
except TypeError as e:
    print(f"  Error TARDIO en el modelo: {e}")
    print("  -> El error se detecto LEJOS de donde entraron los datos")


# --- Enfoque 2: Pydantic ---
print(f"\n{'=' * 55}")
print("  Enfoque 2: Pydantic (falla TEMPRANO)")
print("=" * 55)

try:
    fila_validada = FeatureRow(**datos_malos)
except ValidationError as e:
    print(f"  Error TEMPRANO en la validacion: {e.error_count()} error(es)")
    for err in e.errors():
        print(f"    - Campo: {err['loc']} | Msg: {err['msg']}")
    print("  -> El error se detecto ANTES de llegar al modelo")

# Con datos correctos, Pydantic garantiza tipos
print(f"\n{'=' * 55}")
print("  Enfoque 2: Pydantic con datos correctos")
print("=" * 55)

datos_buenos = {
    "user_id": "u100",
    "age": 28,
    "monthly_income_usd": 4500.0,
    "tenure_months": 12,
}

fila_ok = FeatureRow(**datos_buenos)
score = fake_model_predict(
    age=fila_ok.age,
    income=fila_ok.monthly_income_usd,
    tenure=fila_ok.tenure_months,
)
print(f"  Datos validados: {fila_ok.model_dump()}")
print(f"  Score predicho: {score}")

---

## Ejercicios

Los siguientes ejercicios combinan todos los temas vistos en este notebook. Completa el cuerpo de cada funcion/clase reemplazando `pass` con tu implementacion.

**Tip:** Ejecuta cada celda para verificar que funciona. Los `assert` al final te confirman si la solucion es correcta.

In [ ]:
# ============================================================
# Ejercicio 1: Data Loader Resiliente
# ============================================================
#
# Construi un generador `cargar_registros(datos_crudos)` que:
#
# 1. Recibe una lista de diccionarios (simulando registros JSON).
# 2. Para cada registro:
#    a. Si le falta el campo "user_id" -> loguear WARNING y saltar.
#    b. Si el campo "score" no es numerico -> loguear WARNING y saltar.
#    c. Si el campo "score" es negativo -> levantar DataValidationError.
#    d. Si es valido -> yield el registro.
# 3. Al terminar, loguear INFO con la cantidad de registros procesados.
#
# Usa: generadores, logging, excepciones custom (PipelineError/DataValidationError
# definidos arriba), try/except.
#
# Ejemplo de datos de entrada:
# datos = [
#     {"user_id": "u1", "score": 0.9},   # valido
#     {"score": 0.5},                      # sin user_id -> warning
#     {"user_id": "u3", "score": "abc"},   # score no numerico -> warning
#     {"user_id": "u4", "score": -0.1},    # score negativo -> DataValidationError
#     {"user_id": "u5", "score": 0.7},     # valido
# ]

ej1_logger = logging.getLogger("ejercicio_1")
ej1_logger.setLevel(logging.DEBUG)
ej1_logger.handlers.clear()
ej1_handler = logging.StreamHandler(sys.stdout)
ej1_handler.setFormatter(logging.Formatter("%(levelname)-8s | %(message)s"))
ej1_logger.addHandler(ej1_handler)


def cargar_registros(datos_crudos: list) -> Generator[dict, None, None]:
    """Generador que carga y valida registros de forma resiliente."""
    pass  # <-- Tu implementacion aqui


# --- Prueba ---
# datos_prueba = [
#     {"user_id": "u1", "score": 0.9},
#     {"score": 0.5},
#     {"user_id": "u3", "score": "abc"},
#     {"user_id": "u5", "score": 0.7},
# ]
# resultados = list(cargar_registros(datos_prueba))
# assert len(resultados) == 2, f"Se esperaban 2 validos, se obtuvieron {len(resultados)}"
# print(f"\nEjercicio 1 OK: {len(resultados)} registros validos")

In [ ]:
# ============================================================
# Ejercicio 2: Pydantic - TrainingConfig
# ============================================================
#
# Crea un modelo Pydantic `TrainingConfig` con las siguientes restricciones:
#
# - experiment_name: str, minimo 3 caracteres, maximo 50
# - learning_rate: float, default 0.001, rango (0.0, 1.0) exclusivo
#                  (gt=0, lt=1.0)
# - epochs: int, default 10, rango [1, 1000]
# - batch_size: int, default 32, debe ser potencia de 2
#              (usa field_validator para verificar: n & (n - 1) == 0 y n > 0)
# - optimizer: Literal["adam", "sgd", "adamw"], default "adam"
# - dropout: float, default 0.1, rango [0.0, 0.9]
#
# Despues de definir el modelo:
# 1. Crea una instancia valida y muestrala.
# 2. Intenta crear una invalida (batch_size=48, learning_rate=5.0)
#    y captura el ValidationError.


class TrainingConfig(BaseModel):
    """Configuracion validada para un entrenamiento de modelo ML."""
    pass  # <-- Tu implementacion aqui


# --- Prueba ---
# config_ok = TrainingConfig(experiment_name="churn_v3")
# print(f"Config valida: {config_ok.model_dump()}")
#
# try:
#     TrainingConfig(experiment_name="x", batch_size=48, learning_rate=5.0)
# except ValidationError as e:
#     print(f"\nErrores capturados: {e.error_count()}")
#     for err in e.errors():
#         print(f"  - {err['loc']}: {err['msg']}")
#     print("\nEjercicio 2 OK")

In [ ]:
# ============================================================
# Ejercicio 3: Pipeline completo con generadores + logging +
#              excepciones + Pydantic
# ============================================================
#
# Construi un pipeline que procese un stream de registros de prediccion:
#
# 1. Defini un modelo Pydantic `PredictionRecord`:
#    - request_id: str (min 1 char)
#    - input_text: str (min 10 chars)
#    - confidence: float (0.0 a 1.0)
#
# 2. Defini un generador `validar_stream(records)` que:
#    - Recibe una lista de dicts
#    - Intenta crear un PredictionRecord con cada dict
#    - Si es valido, yield el modelo validado
#    - Si es invalido, loguear WARNING con los errores y continuar
#
# 3. Defini un generador `filtrar_alta_confianza(stream, umbral=0.8)` que:
#    - Recibe el generador anterior
#    - Solo yield registros con confidence >= umbral
#
# 4. Defini una funcion `procesar_pipeline(records, umbral)` que:
#    - Encadena validar_stream → filtrar_alta_confianza
#    - Retorna lista de resultados
#    - Loguea INFO con el total de resultados


# Tu implementacion aqui:
pass


# --- Datos de prueba ---
# registros_prueba = [
#     {"request_id": "r1", "input_text": "analizar este texto de ejemplo", "confidence": 0.95},
#     {"request_id": "r2", "input_text": "otro texto largo suficiente", "confidence": 0.60},
#     {"request_id": "", "input_text": "corto", "confidence": 1.5},  # invalido
#     {"request_id": "r4", "input_text": "texto de prueba completo", "confidence": 0.88},
# ]
#
# resultados = procesar_pipeline(registros_prueba, umbral=0.8)
# assert len(resultados) == 2, f"Se esperaban 2, se obtuvieron {len(resultados)}"
# print(f"\nEjercicio 3 OK: {len(resultados)} registros de alta confianza")

---

## Checklist de consolidacion

Antes de pasar al siguiente notebook, verifica que puedes:

- [ ] Escribir un bloque `try / except / else / finally` completo y explicar cuando se ejecuta cada parte.
- [ ] Crear una jerarquia de excepciones custom con atributos adicionales y usar `raise ... from ...` para encadenar errores.
- [ ] Explicar la diferencia entre `return` y `yield`, y cuando un generador es preferible a una lista.
- [ ] Construir un pipeline de generadores encadenados que procese datos de forma lazy.
- [ ] Usar `sys.getsizeof()` para comparar el consumo de memoria entre una lista y un generador.
- [ ] Elegir correctamente entre comprehension y loop segun el caso (transformacion vs side effects).
- [ ] Configurar un logger con multiples handlers, niveles y formatters (sin usar `print` para diagnostico).
- [ ] Usar lazy formatting (`%s`, `%d`) en lugar de f-strings dentro de llamadas a logging.
- [ ] Definir modelos Pydantic con `Field`, `field_validator` y capturar `ValidationError` de forma estructurada.
- [ ] Integrar excepciones + generadores + logging + Pydantic en un pipeline resiliente.

---

### Proximo paso

**Notebook 04: Programacion Orientada a Objetos (OOP)** - Clases, herencia, protocolos y patrones de diseno para sistemas de ML.

---

*Notebook 03 - Modulo 01 - Python Extra Class - AI Engineering Henry*